1. Kompresja obrazów z użyciem sieci neuronowej  

·         Zbiór danych Fashion-MNIST  

·         Budujemy sieć neuronową dla kompresji danych, a następnie próbujemy sprawdzić czy sieć jest wstanie rozpoznać (poprawnie skwalifikować) dane z dekodera. Proszę również pokazać że sieć do kompresji można „rozciąć” na układ koder-dekoder. 

Tworzymy auto-enkoder i w pierwszej kolejności chcemy pokazać zależność błędu pomiędzy wyjściem a wejściem w stosunku do stopnia kompresji, 

Następnie patrzymy dla jakiś sensownych – wybranych - punktów z powyższej krzywej jak wygląda klasyfikacja danych zakodowanych.        Dla każdego wyniku stosujemy różne miary błędu (ewaluacji) MSE, macierz pomyłek, krzywe ROC,  specyficzność czułość, pole pod ROC itp.  

·         Wyniki proszę porównać z istniejącymi w literaturze.  

In [1]:
import math
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.optimizers import RMSprop,Adam
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Conv2D, Activation, MaxPool2D, Flatten, Dropout, BatchNormalization

# loading dataset
(X, Y), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

print("Train dataset shape:", X.shape, Y.shape)
print("Test dataset shape:", x_test.shape, y_test.shape)

Train dataset shape: (60000, 28, 28) (60000,)
Test dataset shape: (10000, 28, 28) (10000,)


# Uczenie modelu na oryginalnym zbiorze danych

*   Element listy
*   Element listy



In [ ]:
# preprocessing data
X = X / 255.0
x_test = x_test / 255.0
train_size = X.shape[0]
test_size = x_test.shape[0]
img_shape = X.shape[1]

x_train_array = X.reshape(train_size, img_shape, img_shape, 1)
x_test_array = x_test.reshape(test_size, img_shape, img_shape, 1)

In [ ]:
# preprocessing lables
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']
num_classes = len(class_names)
Y_ohe = tf.keras.utils.to_categorical(Y, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [ ]:
# prepering for data for training
validation = 0.2
batch_size = 64
num_epochs = 5
patience = 10

x_train, x_val, y_train, y_val = train_test_split(x_train_array, Y_ohe, test_size=validation)

In [ ]:
# building model
model = tf.keras.Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape=(28,28, 1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dense(10, activation=tf.nn.softmax))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 128)               4

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=patience)

model.compile(optimizer='adam',
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

train_model = model.fit( x_train, y_train,
                  batch_size=batch_size,
                  epochs=num_epochs,
                  verbose=1,
                  validation_data=(x_val, y_val),
                  callbacks=[early_stop])

Epoch 1/5
750/750 [==============================] - 69s 90ms/step - loss: 0.4535 - accuracy: 0.8387 - val_loss: 0.3271 - val_accuracy: 0.8850
Epoch 2/5
750/750 [==============================] - 66s 87ms/step - loss: 0.3022 - accuracy: 0.8912 - val_loss: 0.2720 - val_accuracy: 0.9028
Epoch 3/5
750/750 [==============================] - 66s 88ms/step - loss: 0.2570 - accuracy: 0.9063 - val_loss: 0.2661 - val_accuracy: 0.9032
Epoch 4/5
750/750 [==============================] - 66s 87ms/step - loss: 0.2240 - accuracy: 0.9178 - val_loss: 0.2474 - val_accuracy: 0.9133
Epoch 5/5
750/750 [==============================] - 70s 94ms/step - loss: 0.2002 - accuracy: 0.9260 - val_loss: 0.2423 - val_accuracy: 0.9131


In [ ]:
score = model.evaluate(x_test_array, y_test, steps=math.ceil(10000/32))
# checking the test loss and test accuracy
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 5s 15ms/step - loss: 0.2670 - accuracy: 0.9041
Test loss: 0.26704004406929016
Test accuracy: 0.9041000008583069


In [ ]:
def make_and_fit():
    inputs = Input(shape=(28, 28, 1))

    en_conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    en_conv1 = BatchNormalization()(en_conv1)
    en_conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(en_conv1)
    en_conv1 = BatchNormalization()(en_conv1)
    en_pool1 = MaxPooling2D(pool_size=(2, 2))(en_conv1)
    en_conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(en_pool1)
    en_conv2 = BatchNormalization()(en_conv2)
    en_conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(en_conv2)
    en_conv2 = BatchNormalization()(en_conv2)
    en_pool2 = MaxPooling2D(pool_size=(2, 2))(en_conv2)
    en_conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(en_pool2)
    en_conv3 = BatchNormalization()(en_conv3)
    en_conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(en_conv3)
    encoded = BatchNormalization()(en_conv3)
    
    encoder = Model(inputs=inputs, outputs=encoded)
    
    de_conv1 = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
    de_conv1 = BatchNormalization()(de_conv1)
    de_conv1 = Conv2D(128, (3, 3), activation='relu', padding='same')(de_conv1)
    de_conv1 = BatchNormalization()(de_conv1)
    up1 = UpSampling2D((2, 2))(de_conv1)
    de_conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
    de_conv2 = BatchNormalization()(de_conv2)
    de_conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(de_conv2)
    de_conv2 = BatchNormalization()(de_conv2)
    up2 = UpSampling2D((2, 2))(de_conv2)
    de_conv3 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
    de_conv3 = BatchNormalization()(de_conv3)
    de_conv3 = Conv2D(32, (3, 3), activation='relu', padding='same')(de_conv3)
    de_conv3 = BatchNormalization()(de_conv3)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(de_conv3)
    decoder = Model(inputs=encoded, outputs=decoded)
    
    x = encoder(inputs)
    x = decoder(x)
    model = Model(inputs=inputs, outputs=x)
    model.compile(optimizer=optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy', 'mse'])

    print(model.summary())
    
    clr = ReduceLROnPlateau(
        monitor='loss',
        factor=0.5,
        patience=3,
        min_delta=0.01,
        cooldown=0,
        min_lr=1e-7,
        verbose=1)

    model.fit(
        x_train,
        x_train,
        batch_size=256,
        epochs=50,
        shuffle=True,
        validation_data=(x_test, x_test),
        callbacks=[clr])

    return model, encoder, decoder

model, encoder, decoder = make_and_fit()